# Finite Volume Method applied to 2D Heat Question

While the finite difference method is a popular choice for solving partial differential equations on regular grids, the finite volume method is often preferred for irregular grids. In this notebook, we will apply the finite volume method to solve the 2D heat equation on an irregular grid (admittedly, a very simple one which the FDM could handle just as well).

The heat equation is given by

$$\frac{du}{dt} = \alpha \nabla^2 u$$

where $u$ is the temperature, $t$ is time, and $\alpha$ is the thermal diffusivity.

The finite volume method is based on the idea of integrating the differential equation over small control volumes. The heat equation can be written in integral form as:

$$\int_{V} \frac{du}{dt} dV = \alpha \int_{V} \nabla^2 u dV$$

where $V$ is the control volume. Applying the divergence theorem to the right-hand side, we get:

$$\int_{V} \frac{du}{dt} dV = \alpha \int_{\partial V} \nabla u \cdot \mathbf{\hat{n}} dA$$

where $\partial V$ is the boundary of the control volume, and $\mathbf{\hat{n}}$ is the outward normal vector. The integral on the right-hand side can be approximated by the fluxes through the faces of the control volume. The finite volume method then discretizes the domain into a set of control volumes, and the heat equation is solved for each control volume. This looks like:

$$\frac{du_i}{dt} = \frac{1}{V_i} \sum_{j=1}^{N} \alpha (u_j - u_i) A_{ij}$$

where $u_i$ is the temperature in control volume $i$, $V_i$ is the volume of control volume $i$, $N$ is the number of neighboring control volumes, $\alpha_{ij}$ is the diffusion coefficient between control volumes $i$ and $j$, and $A_{ij}$ is the area of the face between control volumes $i$ and $j$.

$$ \frac{1}{\Delta t} (u_i^{n+1} - u_i^{n}) V_i = \alpha \sum_{\text{faces } j} (u_j - u_i) \frac{A_j}{d_j} $$

Solving for $u_i^{n}$ gives:

$$u_i^n = u_i^{n+1} - \frac{\Delta t \alpha}{V_i} \sum_{\text{faces}\ j} (u_j^{n+1} - u_i^{n+1}) \frac{A_{ij}}{d_{ij}}$$

where $d_{ij}$ is the distance between the centroids of control volumes $i$ and $j$.

In [ ]:
import numpy as np

# Create a mesh of points to run our simulation on.

def thick_sine_wave(x, y, period=0.5, amplitude=0.1, thickness=0.1, y_offset=0.5, x_min=0.1, x_max=0.9):
    # Create a thick sine wave.
    if not x_min < x < x_max:
        return False 
    z = y_offset + amplitude * np.sin(2 * np.pi * x / period)
    if y - thickness < z < y + thickness:
        return True
    return False

def torus(x,y, R=0.42, r=0.2, x_offset=0.5, y_offset=0.5):
    # Create a torus.
    dx = x - x_offset
    dy = y - y_offset

    dist = np.sqrt(dx**2 + dy**2)
    if r < dist < R:
        return True
    return False

grid_x, grid_y = np.meshgrid(*[np.linspace(0, 1, 100)] * 2)
m = np.zeros((100, 100), dtype=bool)

for i in range(100):
    for j in range(100):
        m[i, j] = torus(grid_x[i][j], grid_y[i][j])

from matplotlib import pyplot as plt
plt.imshow(m, cmap='gray', origin='lower')

In [ ]:
import math

class Mesh:
    def __init__(self, verts, edges):
        self.verticies = verts
        self.edges = edges

    def vertex_count(self):
        return len(self.verticies)
    
def construct_triangular_mesh(shape_function, bounds=((0,1),(0,1)), resolution = 0.1):
    # Create a triangular grid of points.
    x_min, x_max = bounds[0]
    y_min, y_max = bounds[1]

    traingular_mesh_points = []

    y = y_min
    x_offset = 0

    while True:
        for x in np.arange(x_min, x_max, resolution):
            if x_min < x < x_max and y_min < y < y_max:
                if shape_function(x, y):
                    traingular_mesh_points.append((x + x_offset, y))

        y += math.sqrt(3) / 2 * resolution
        x_offset = (x_offset + (resolution / 2)) % resolution

        if y > y_max:
            break

    edges = [] # (index1, index2)
    for i, vertex1 in enumerate(traingular_mesh_points):
        for j, vertex2 in enumerate(traingular_mesh_points):
            if i != j:
                if np.linalg.norm(np.array(vertex1) - np.array(vertex2)) < (resolution * 1.5) + 1e-6:
                    edges.append((i, j))

    return Mesh(traingular_mesh_points, edges)

mesh = construct_triangular_mesh(torus, bounds=((0,1),(0,1)), resolution = 0.02)
print(f"Made Torus mesh with vertex count: {mesh.vertex_count()}")

In [ ]:
# Plot mesh using matplotlib
plt.figure(figsize=(10,10))
for edge in mesh.edges:
    plt.plot([mesh.verticies[edge[0]][0], mesh.verticies[edge[1]][0]], [mesh.verticies[edge[0]][1], mesh.verticies[edge[1]][1]], 'black')

verts = np.array(mesh.verticies)
plt.plot(verts[:,0], verts[:,1], 'ro')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title("Triangular Mesh of Torus")

plt.show()

In [ ]:
from matplotlib.collections import LineCollection

mesh_temps = np.zeros(mesh.vertex_count())

heat_centers = [(0.2, 0.8), (0.7, 0.4)]
heat_radiuses = [0.2, 0.2]

for heat_center, heat_radius in zip(heat_centers, heat_radiuses):
    for i, vertex in enumerate(mesh.verticies):
        if np.linalg.norm(np.array(vertex) - np.array(heat_center)) < heat_radius:
            mesh_temps[i] = 1

def show_mesh(mesh, temps, save_path=None, title=""):
    plt.figure(figsize=(12,10))
    edge_lines = []
    for edge in mesh.edges:
        edge_lines.append([mesh.verticies[edge[0]], mesh.verticies[edge[1]]])
                          
    lc = LineCollection(edge_lines, linewidths=1, zorder=-100, color='black')
    plt.gca().add_collection(lc)

    verts = np.array(mesh.verticies)
    # plot the temperatures
    plt.scatter(verts[:,0], verts[:,1], c=temps, cmap='coolwarm', vmin=0, vmax=1, s=60)
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.colorbar()
    if title != None:
        plt.title(title)

    if save_path:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()

ground_truth_total_energy = mesh_temps.sum()
show_mesh(mesh, mesh_temps, title="Initial State")

# 

$$u_i^n = u_i^{n+1} - \frac{\Delta t \alpha}{V_i} \sum_{\text{faces}\ j} (u_j^{n+1} - u_i^{n+1}) \frac{A_{ij}}{d_{ij}}$$

In [ ]:
import math

DELTA_T = 0.002
RESOLUTION = 0.023
VOLUME = RESOLUTION * RESOLUTION * math.sqrt(3) / 4
DISTANCE = RESOLUTION
CONDUCTIVITY = 0.01

def get_A_matrix(mesh):
    A = np.zeros((mesh.vertex_count(), mesh.vertex_count()))
    r = DELTA_T * (CONDUCTIVITY / VOLUME) * DISTANCE / DISTANCE
    # init diagonal
    for i in range(mesh.vertex_count()):
        A[i, i] = 1
    for edge in mesh.edges:
        i, j = edge
        A[i, j] = -r
        A[i, i] += r
    return A

get_A_matrix(mesh)

In [ ]:
import os

from tqdm import tqdm
from scipy.sparse.linalg import bicgstab

if not os.path.exists('output'):
    os.makedirs('output')

A = get_A_matrix(mesh)
def solve_heat(temps):
    b = temps.copy()
    temps, _ = bicgstab(A, b)
    return temps

for i in tqdm(range(1000)):
    energy = mesh_temps.sum()
    error_perc = abs(energy - ground_truth_total_energy) / ground_truth_total_energy
    error_perc = f'{error_perc:.2%}'
    show_mesh(mesh, mesh_temps, save_path=f'output/{i:04d}.png', title=f'Iteration: {i} | Energy Error: {error_perc}')
    
    mesh_temps = solve_heat(mesh_temps)

show_mesh(mesh, mesh_temps)

In [ ]:
# Use ffmpeg to create a video. Start with making the intro using only the first frame for 3 seconds
!ffmpeg -y -framerate 0.3 -i output/0000.png -c:v libx264 -r 30 -pix_fmt yuv420p output/intro.mp4
# Then create the video
!ffmpeg -y -framerate 30 -i output/%04d.png -c:v libx264 -r 30 -pix_fmt yuv420p output/animation.mp4
# Concatenate the intro and the video
!ffmpeg -y -i output/intro.mp4 -i output/animation.mp4 -filter_complex "[0:v] [1:v] concat=n=2:v=1 [v]" -map "[v]" output/output.mp4